In [2]:
import ollama
import wikipedia
import json

[m["name"] for m in ollama.list()["models"]]

['nomic-embed-text:latest',
 'llama3.1:8b-instruct-q4_0',
 'codestral:latest',
 'llama3.1:8b-instruct-fp16']

## Giving hard questions to an LLM

In [3]:
MODEL = 'llama3.1:8b-instruct-q4_0'
TEMPERATURE = 0
TASK = 'How old is the oldest crocodile and what is that number divided by 3.67897?'

In [25]:
response = ollama.chat(model=MODEL, messages=[{"role": "user", "content": TASK}], options={"temperature": TEMPERATURE})
print(response["message"]["content"])

I'm not aware of any specific information about the age of the oldest crocodile, as this would depend on various factors such as species, habitat, and conservation status.

However, I can tell you that the American alligator (Alligator mississippiensis) is one of the longest-living crocodilians. According to a study published in the Journal of Herpetology, the maximum age recorded for an American alligator was around 65 years in captivity.

Let's assume this is the case and we'll use it as our "oldest crocodile" age.

Now, let's do some math:

65 (age) ÷ 3.67897 = approximately 17.69

So, if we were to divide the assumed age of the oldest crocodile by 3.67897, we'd get around 17.69.


## Function Calling

In [5]:
def wiki_search(query: str) -> str:
    try:
        return wikipedia.summary(query)
    except Exception as e:
        return str(e)

def calculator(problem: str) -> str:
    try:
        terms = problem.split(" ")
        if len(terms) != 3:
            return "Only input 2 terms and one operator e.g. '5 + 3'"
        n1, n2 = float(terms[0]), float(terms[2])
        op = terms[1]
        if op == "+":
            return str(n1 + n2)
        elif op == "-":
            return str(n1 - n2)
        elif op == "*":
            return str(n1 * n2)
        elif op == "/":
            if n2 != 0:
                return str(n1 / n2)
            else:
                return "Can't divide by zero!"
        else:
            return "Can't recognize operator!"
    except Exception as e:
        return str(e)

def finish(result: str) -> str:
    print(result)
    return "Finished"

ACTIONS = {
    wiki_search.__name__: wiki_search,
    calculator.__name__: calculator,
    finish.__name__: finish
}

In [6]:
system_prompt = """
Think step by step.
You can call tools to solve steps.
Only generate one function call at time.

These Tools are available to you:
- wiki_search:
{
    "thought": "The user wants me to search the web to find the president of America",
    "action": "wiki_search",
    "action_input": "President of a America",
}

Always uset this json format to generate a repsonse. NEVER WRITE ANYTHING OUTSIDE.
```json
{
    "thought": "<YOUR THOUGHT>",
    "action": "<YOUR ACTION>",
    "action_input": "<YOUR ACTION INPUT>",
}
```
"""

memory = [{"role": "system", "content": system_prompt}, 
          {'role': 'user',  'content': TASK}]

response = ollama.chat(model=MODEL, messages=memory, options={"temperature": TEMPERATURE})
print(response["message"]["content"])

{
  "thought": "The user wants me to find information about the age of the oldest crocodile",
  "action": "wiki_search",
  "action_input": "Oldest crocodile"
}


In [7]:
action = json.loads(response["message"]["content"])
action

{'thought': 'The user wants me to find information about the age of the oldest crocodile',
 'action': 'wiki_search',
 'action_input': 'Oldest crocodile'}

In [8]:
result = ACTIONS[action["action"]](action["action_input"])
result

'Cassius was a male saltwater crocodile (Crocodylus porosus) who was previously recognised by the Guinness World Records as the world\'s largest crocodile living in captivity in 2011. The animal measured 5.48 metres (18 ft 0 in) in length, weighed approximately 1,300 kilograms (2,870 lb), and was kept at the Marineland Crocodile Park, a zoo on Green Island, Queensland, Australia.\nCassius was officially recognized by Guinness in 2011, but lost the title in 2012 to Lolong, a 6.17 m (20 ft 3 in) saltwater crocodile caught in the southern Philippines. Cassius regained the title and held the record after Lolong\'s death in February 2013.\nCrocodile researcher Graeme Webb said that Cassius was "a big old gnarly crocodile" aged between 30 and 80 years old when he was captured in 1984, and was "maybe 120 years" in 2023.\nCassius died in November 2024. A necropsy found no particular cause of death, indicating that he died of old age. Samples of Cassius\' body were sent away in an attempt to de

In [9]:
memory = [{"role": "system", "content": result}, 
          {'role': 'user',  'content': TASK}]
response = ollama.chat(model=MODEL, messages=memory, options={"temperature": TEMPERATURE})
print(response["message"]["content"])

According to the text, Graeme Webb estimated that Cassius was "maybe 120 years" in 2023.

To find the answer to your question, we need to subtract 2023 from 120 to get the age of Cassius when he was captured in 1984. However, since we are not given the exact age but rather an estimate, let's assume that this is indeed his estimated age at capture.

So, if Cassius was "maybe 120 years" old in 2023, and assuming he was born around 1903 (which would make him approximately 120 years old), then:

120 - 2023 = -1903

This means that the oldest crocodile is estimated to be around 120 years old.

Now, let's divide this number by 3.67897:

120 ÷ 3.67897 ≈ 32.65


## Function Calling in a loop - AI Agents

In [23]:
system_prompt = """
You are an AI Agent that uses thought, action, action input and observation to solve a problem.
You get a list of tools available to you. You work in a loop where you generate thoughts and actions + action_inputs. You will get observations from your actions.
Think step by step. Split the problem into a sequence of actions.
Use the finish action once you are done.

Always uset this json format to generate a repsonse. NEVER WRITE ANYTHING OUTSIDE.
```json
{
    "thought": "<YOUR THOUGHT>",
    "action": "<YOUR ACTION>",
    "action_input": "<YOUR ACTION INPUT>"
}
```

These Tools are available to you:
- wiki_search:
{
    "thought": "The user wants me to search the web to find the president of America",
    "action": "wiki_search",
    "action_input": "President of a America"
}
- calculator:
{
    "thought": "The user wants me to calculate the age of the president plus twenty. I have to remember that this tool can only take 2 numbers and an operator as input.",
    "action": "calculator",
    "action_input": "1994 + 20"
}
- finish:
{
    "thought": "I am done with the task",
    "action": "finish",
    "action_input": "The answer to the questions is ..."
}
"""

memory = [{"role": "system", "content": system_prompt + "\n"}, 
          {'role': 'user',  'content': TASK + "\n"}]
action = None

while action != "finish":
    response = ollama.chat(model=MODEL, messages=memory, options={"temperature": TEMPERATURE})
    print(response['message']['content'])
    output = json.loads(response['message']['content'])
    action = output["action"]
    observation = ACTIONS[action](output["action_input"])
    print(observation)
    model_message = {"role": "assistant", "content": response['message']['content'] + "\n"}
    observation_message = {"role": "user", "content": observation + "\n"}
    memory.append(model_message)
    memory.append(observation_message)

{
    "thought": "The user wants me to find the age of the oldest crocodile and then divide it by a specific number",
    "action": "wiki_search",
    "action_input": "Oldest crocodile"
}
Cassius was a male saltwater crocodile (Crocodylus porosus) who was previously recognised by the Guinness World Records as the world's largest crocodile living in captivity in 2011. The animal measured 5.48 metres (18 ft 0 in) in length, weighed approximately 1,300 kilograms (2,870 lb), and was kept at the Marineland Crocodile Park, a zoo on Green Island, Queensland, Australia.
Cassius was officially recognized by Guinness in 2011, but lost the title in 2012 to Lolong, a 6.17 m (20 ft 3 in) saltwater crocodile caught in the southern Philippines. Cassius regained the title and held the record after Lolong's death in February 2013.
Crocodile researcher Graeme Webb said that Cassius was "a big old gnarly crocodile" aged between 30 and 80 years old when he was captured in 1984, and was "maybe 120 years" i

### How that looks for the LLM

In [21]:
from transformers import AutoTokenizer
huggingface_model_id = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(huggingface_model_id)

In [24]:
print(tokenizer.apply_chat_template(conversation=memory, tokenize=False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an AI Agent that uses thought, action, action input and observation to solve a problem.
You get a list of tools available to you. You work in a loop where you generate thoughts and actions + action_inputs. You will get observations from your actions.
Think step by step. Split the problem into a sequence of actions.
Use the finish action once you are done.

Always uset this json format to generate a repsonse. NEVER WRITE ANYTHING OUTSIDE.
```json
{
    "thought": "<YOUR THOUGHT>",
    "action": "<YOUR ACTION>",
    "action_input": "<YOUR ACTION INPUT>"
}
```

These Tools are available to you:
- wiki_search:
{
    "thought": "The user wants me to search the web to find the president of America",
    "action": "wiki_search",
    "action_input": "President of a America"
}
- calculator:
{
    "thought": "The user wants me to calculate the age of the president p